In [35]:
import pandas as pd
from sunpy.coordinates import HeliographicStonyhurst, Helioprojective
import astropy.units as u
from astropy.coordinates import SkyCoord
from sunpy.net import Fido
from sunpy.net import attrs as a
from sunpy.map import Map
from sunpy.coordinates import frames, get_body_heliographic_stonyhurst, get_horizons_coord, transform_with_sun_center
import matplotlib.pyplot as plt
import numpy as np

In [5]:
df1 = pd.read_csv('010121-130223_flare_location_output_part1.csv', index_col=0) # change name accordingly
df2 = pd.read_csv('010121-130223_flare_location_output_part2.csv', index_col=0)

In [6]:
df1.head()

,flare ID,request ID,X flare loc,Y flare loc,FWDFIT X SIGMA,FWDFIT Y SIGMA,MAX X COORD BP,MAX Y COORD BP
0,2101061410,2101060003,-37.762600,-435.479000,2.529080,2.698680,-37.076300,-437.500000
185,2101250635,2101250007,-1642.337158,676.563354,0.831036,1.331697,-1651.377075,667.769958
186,2102132346,2102130001,644.262634,715.233521,1.172359,0.926795,637.098267,705.708862
187,2102140144,2102140006,618.121277,706.836792,0.826290,0.892662,617.354980,705.548523
188,2102140529,2102140007,703.509338,727.776001,1.512266,1.221617,695.439880,724.824646


In [148]:
df2.head()

,flare ID,STIX MAX X BP COORD,STIX MAX Y BP COORD,time shift
0,2101061410,-485.956000,84.943900,162.883695
186,2102132346,616.684387,-656.847412,244.781060
187,2102140144,617.395752,-637.926208,244.732898
188,2102140529,637.068359,-714.557434,244.637013
189,2102140807,636.149841,-753.023865,244.570256


In [7]:
len(df1), len(df2)

(9446, 9446)

In [8]:
df2['flare ID'].nunique(), df2['flare ID'].nunique()

(9446, 9446)

In [96]:
df1['flare ID'].nunique(), df2['flare ID'].nunique()

(9446, 9446)

In [9]:
new_df = df2.join(df1.set_index('flare ID'), on='flare ID')

In [10]:
len(new_df)

9446

In [11]:
new_df.head()

,flare ID,STIX MAX X BP COORD,STIX MAX Y BP COORD,time shift,request ID,X flare loc,Y flare loc,FWDFIT X SIGMA,FWDFIT Y SIGMA,MAX X COORD BP,MAX Y COORD BP
0,2101061410,-485.956000,84.943900,162.883695,2101060003,-37.762600,-435.479000,2.529080,2.698680,-37.076300,-437.500000
185,2101250635,673.040588,1664.330566,221.886786,2101250007,-1642.337158,676.563354,0.831036,1.331697,-1651.377075,667.769958
186,2102132346,616.684387,-656.847412,244.781060,2102130001,644.262634,715.233521,1.172359,0.926795,637.098267,705.708862
187,2102140144,617.395752,-637.926208,244.732898,2102140006,618.121277,706.836792,0.826290,0.892662,617.354980,705.548523
188,2102140529,637.068359,-714.557434,244.637013,2102140007,703.509338,727.776001,1.512266,1.221617,695.439880,724.824646


In [12]:
full_fl_paths = pd.read_csv('full_flarelist_all_with_paths.csv')

In [13]:
full_fl_paths.head()

,flare_id,att_in,GOES_flux,start_UTC,peak_UTC,end_UTC,4-10 keV,10-15 keV,number_available_files,available_file_request_IDs,solo_lat,solo_lon,solo_AU_distance,15-25 keV,25-50 keV,50-84 keV,GOES_class,Request IDs,aux_paths,pixel_paths
0,2101061410,False,1.325816e-08,2021-01-06T14:08:15.464,2021-01-06 14:10:47.465,2021-01-06T14:21:31.466,671,151,1,[2101060003],-1.695356,135.627606,0.656878,99,991,543,A,2101060003,./aux_data/solo_L2_stix-aux-ephemeris_20210106...,./pixel_data/solo_L1_stix-sci-xray-cpd_2021010...
1,2101250635,False,5.092464e-07,2021-01-25T06:32:51.072,2021-01-25 06:35:31.073,2021-01-25T06:41:51.073,607,91,1,[2101250007],-0.525180,158.879081,0.539705,99,927,543,B5.1,2101250007,./aux_data/solo_L2_stix-aux-ephemeris_20210125...,./pixel_data/solo_L1_stix-sci-xray-cpd_2021012...
2,2102132346,False,1.535086e-08,2021-02-13T23:45:06.658,2021-02-13 23:46:18.658,2021-02-13T23:49:54.659,607,83,1,[2102130001],1.445369,-160.187451,0.496872,91,927,543,A,2102130001,./aux_data/solo_L2_stix-aux-ephemeris_20210213...,./pixel_data/solo_L1_stix-sci-xray-cpd_2021021...
3,2102140144,False,1.557456e-08,2021-02-14T01:41:06.670,2021-02-14 01:44:14.670,2021-02-14T01:49:14.671,1983,463,1,[2102140006],1.451802,-160.007020,0.496985,183,927,543,A,2102140006,./aux_data/solo_L2_stix-aux-ephemeris_20210214...,./pixel_data/solo_L1_stix-sci-xray-cpd_2021021...
4,2102140529,False,1.816594e-08,2021-02-14T05:23:46.693,2021-02-14 05:29:06.693,2021-02-14T05:36:18.694,671,83,1,[2102140007],1.463975,-159.663318,0.497207,83,927,543,A,2102140007,./aux_data/solo_L2_stix-aux-ephemeris_20210214...,./pixel_data/solo_L1_stix-sci-xray-cpd_2021021...


In [14]:
len(full_fl_paths)

11983

In [15]:
final_df = pd.merge(full_fl_paths, new_df, right_on ='flare ID', left_on = 'flare_id')

In [16]:
len(final_df)

9446

In [17]:
final_df.head()

,flare_id,att_in,GOES_flux,start_UTC,peak_UTC,end_UTC,4-10 keV,10-15 keV,number_available_files,available_file_request_IDs,...,STIX MAX X BP COORD,STIX MAX Y BP COORD,time shift,request ID,X flare loc,Y flare loc,FWDFIT X SIGMA,FWDFIT Y SIGMA,MAX X COORD BP,MAX Y COORD BP
0,2101061410,False,1.325816e-08,2021-01-06T14:08:15.464,2021-01-06 14:10:47.465,2021-01-06T14:21:31.466,671,151,1,[2101060003],...,-485.956000,84.943900,162.883695,2101060003,-37.762600,-435.479000,2.529080,2.698680,-37.076300,-437.500000
1,2101250635,False,5.092464e-07,2021-01-25T06:32:51.072,2021-01-25 06:35:31.073,2021-01-25T06:41:51.073,607,91,1,[2101250007],...,673.040588,1664.330566,221.886786,2101250007,-1642.337158,676.563354,0.831036,1.331697,-1651.377075,667.769958
2,2102132346,False,1.535086e-08,2021-02-13T23:45:06.658,2021-02-13 23:46:18.658,2021-02-13T23:49:54.659,607,83,1,[2102130001],...,616.684387,-656.847412,244.781060,2102130001,644.262634,715.233521,1.172359,0.926795,637.098267,705.708862
3,2102140144,False,1.557456e-08,2021-02-14T01:41:06.670,2021-02-14 01:44:14.670,2021-02-14T01:49:14.671,1983,463,1,[2102140006],...,617.395752,-637.926208,244.732898,2102140006,618.121277,706.836792,0.826290,0.892662,617.354980,705.548523
4,2102140529,False,1.816594e-08,2021-02-14T05:23:46.693,2021-02-14 05:29:06.693,2021-02-14T05:36:18.694,671,83,1,[2102140007],...,637.068359,-714.557434,244.637013,2102140007,703.509338,727.776001,1.512266,1.221617,695.439880,724.824646


In [18]:
final_df.keys()

Index(['flare_id', 'att_in', 'GOES_flux', 'start_UTC', 'peak_UTC', 'end_UTC',
       '4-10 keV', '10-15 keV', 'number_available_files',
       'available_file_request_IDs', 'solo_lat', 'solo_lon',
       'solo_AU_distance', '15-25 keV', '25-50 keV', '50-84 keV', 'GOES_class',
       'Request IDs', 'aux_paths', 'pixel_paths', 'flare ID',
       'STIX MAX X BP COORD', 'STIX MAX Y BP COORD', 'time shift',
       'request ID', 'X flare loc', 'Y flare loc', 'FWDFIT X SIGMA',
       'FWDFIT Y SIGMA', 'MAX X COORD BP', 'MAX Y COORD BP'],
      dtype='object')

In [28]:
final_df.to_csv('010121-130223_near_final.csv')

In [42]:
#cannot make more than 5498 requests so have to do separately and append lists later if dataframe is too large

earth_x_helioprojective = []
earth_y_helioprojective = []

earth_x_heliographic = []
earth_y_heliographic = []

for i in range(5498,len(final_df)):
    x = final_df.iloc[i]['X flare loc']
    y = final_df.iloc[i]['Y flare loc']
    obstime = final_df.iloc[i]['peak_UTC'] 
    solo_pos = get_horizons_coord("solar orbiter",obstime)
    earth_coord_helioprojective = SkyCoord(x*u.arcsec, y*u.arcsec, frame=Helioprojective, observer=solo_pos).transform_to(Helioprojective(observer="earth", obstime=obstime))
    earth_coord_heliocentric = SkyCoord(x*u.arcsec, y*u.arcsec, frame=Helioprojective, observer=solo_pos).transform_to(HeliographicStonyhurst(obstime=obstime))
    
    earth_x_heliographic.append(earth_coord_heliocentric.lon.value)
    earth_y_heliographic.append(earth_coord_heliocentric.lat.value)
    
    earth_x_helioprojective.append(earth_coord_helioprojective.Tx.value)
    earth_y_helioprojective.append(earth_coord_helioprojective.Ty.value)
    print(i)

INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5498
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5499


INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5500
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5501
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5502
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5503
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5504
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5505
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5506
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5507
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5508
INFO: Obtained JPL 

INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5576
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5577
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5578
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5579
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5580
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5581
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5582
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5583
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5584
INFO: Obtained JPL 

INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5652
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5653
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5654
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5655
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5656
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5657
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5658
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5659
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5660
INFO: Obtained JPL 

INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5728
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5729
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5730
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5731
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5732
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5733
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5734
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5735
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
5736
INFO: Obtained JPL 

...

INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9085
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9086
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9087
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9088
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9089
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9090
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9091
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9092
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9093
INFO: Obtained JPL 

INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9161
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9162
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9163
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9164
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9165
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9166
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9167
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9168
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9169
INFO: Obtained JPL 

INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9237
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9238
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9239
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9240
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9241
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9242
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9243
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9244
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9245
INFO: Obtained JPL 

INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9313
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9314
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9315
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9316
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9317
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9318
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9319
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9320
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9321
INFO: Obtained JPL 

INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9389
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9390
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9391
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9392
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9393
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9394
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9395
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9396
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
9397
INFO: Obtained JPL 

In [43]:
earth_x_helioprojective_5498_on = earth_x_helioprojective
earth_y_helioprojective_5498_on = earth_y_helioprojective
earth_x_heliographic_5498_on = earth_x_heliographic
earth_y_heliographic_5498_on = earth_y_heliographic

In [44]:
d = {'earth_x_helioprojective_5498_on':earth_x_helioprojective_5498_on,'earth_y_helioprojective_5498_on':earth_y_helioprojective_5498_on,
    'earth_x_heliographic_5498_on':earth_x_heliographic_5498_on,'earth_y_heliographic_5498_on':earth_y_heliographic_5498_on }


In [45]:
temp = pd.DataFrame(d)

In [46]:
temp.to_csv('temp_storage_of_earth_locations_5498_on.csv')

In [25]:
# run these few cells when looping from index 0 to 5498
earth_x_helioprojective_first_5498 = earth_x_helioprojective
earth_y_helioprojective_first_5498 = earth_y_helioprojective
earth_x_heliographic_first_5498 = earth_x_heliographic
earth_y_heliographic_first_5498 = earth_y_heliographic

In [30]:
d = {'earth_x_helioprojective_first_5498':earth_x_helioprojective_first_5498,'earth_y_helioprojective_first_5498':earth_y_helioprojective_first_5498,
    'earth_x_heliographic_first_5498':earth_x_heliographic_first_5498,'earth_y_heliographic_first_5498':earth_y_heliographic_first_5498 }

In [31]:
temp = pd.DataFrame(d)

In [33]:
temp.to_csv('temp_storage_of_earth_locations_first_5498.csv')

In [50]:
first = pd.read_csv('temp_storage_of_earth_locations_first_5498.csv', index_col=0)
second = pd.read_csv('temp_storage_of_earth_locations_5498_on.csv', index_col=0)

In [51]:
first.head()

,earth_x_helioprojective_first_5498,earth_y_helioprojective_first_5498,earth_x_heliographic_first_5498,earth_y_heliographic_first_5498
0,660.921841,-356.033488,134.072058,-18.944095
1,855.563671,396.705104,71.481742,22.347076
2,-583.288612,293.652991,-139.159952,22.861221
3,-575.454512,288.716023,-139.906750,22.613293
4,-612.172284,303.481826,-136.497375,23.276099


In [69]:
final_df["Lon Heliographic"] = np.asarray(first['earth_x_heliographic_first_5498'].append(second['earth_x_heliographic_5498_on'])
                                         )

final_df["Lat Heliographic"] = np.asarray(first['earth_y_heliographic_first_5498'].append(second['earth_y_heliographic_5498_on'])
                                         )

final_df["Earth X Helioprojective"] = np.asarray(first['earth_x_helioprojective_first_5498'].append(second['earth_x_helioprojective_5498_on']) 
                                                )

final_df['Earth Y Helioprojective'] = np.asarray(first['earth_y_helioprojective_first_5498'].append(second['earth_y_helioprojective_5498_on']) 
                                                )


/var/folders/l6/88j7fvx51715f93rw6wngs_m0000gn/T/ipykernel_1098/2337806416.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df["Lon Heliographic"] = np.asarray(first['earth_x_heliographic_first_5498'].append(second['earth_x_heliographic_5498_on'])
/var/folders/l6/88j7fvx51715f93rw6wngs_m0000gn/T/ipykernel_1098/2337806416.py:4: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df["Lat Heliographic"] = np.asarray(first['earth_y_heliographic_first_5498'].append(second['earth_y_heliographic_5498_on'])
/var/folders/l6/88j7fvx51715f93rw6wngs_m0000gn/T/ipykernel_1098/2337806416.py:7: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df["Earth X Helioprojective"] = np.asarray(first['earth_x_helioprojective_first_549

In [70]:
final_df

,flare_id,att_in,GOES_flux,start_UTC,peak_UTC,end_UTC,4-10 keV,10-15 keV,number_available_files,available_file_request_IDs,...,X flare loc,Y flare loc,FWDFIT X SIGMA,FWDFIT Y SIGMA,MAX X COORD BP,MAX Y COORD BP,Lon Heliographic,Lat Heliographic,Earth X Helioprojective,Earth Y Helioprojective
0,2101061410,False,1.325816e-08,2021-01-06T14:08:15.464,2021-01-06 14:10:47.465,2021-01-06T14:21:31.466,671,151,1,[2101060003],...,-37.762600,-435.479000,2.529080,2.698680,-37.076300,-437.500000,134.072058,-18.944095,660.921841,-356.033488
1,2101250635,False,5.092464e-07,2021-01-25T06:32:51.072,2021-01-25 06:35:31.073,2021-01-25T06:41:51.073,607,91,1,[2101250007],...,-1642.337158,676.563354,0.831036,1.331697,-1651.377075,667.769958,71.481742,22.347076,855.563671,396.705104
2,2102132346,False,1.535086e-08,2021-02-13T23:45:06.658,2021-02-13 23:46:18.658,2021-02-13T23:49:54.659,607,83,1,[2102130001],...,644.262634,715.233521,1.172359,0.926795,637.098267,705.708862,-139.159952,22.861221,-583.288612,293.652991
3,2102140144,False,1.557456e-08,2021-02-14T01:41:06.670,2021-02-14 01:44:14.670,2021-02-14T01:49:14.671,1983,463,1,[2102140006],...,618.121277,706.836792,0.826290,0.892662,617.354980,705.548523,-139.906750,22.613293,-575.454512,288.716023
4,2102140529,False,1.816594e-08,2021-02-14T05:23:46.693,2021-02-14 05:29:06.693,2021-02-14T05:36:18.694,671,83,1,[2102140007],...,703.509338,727.776001,1.512266,1.221617,695.439880,724.824646,-136.497375,23.276099,-612.172284,303.481826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9441,2302122101,False,3.943743e-06,2023-02-12T20:58:03.713,2023-02-12 21:01:03.713,2023-02-12T21:12:03.715,1343,107,1,[2302124344],...,-285.577911,227.744308,0.613444,0.537331,-289.806030,227.298859,-47.140964,11.104499,-701.052398,262.464843
9442,2302122201,False,2.654562e-06,2023-02-12T21:52:43.719,2023-02-12 22:01:51.720,2023-02-12T22:10:27.721,607,75,1,[2302124344],...,-28.208405,927.354004,6.059155,6.355738,-34.104820,920.830078,-34.666643,55.083800,-316.903348,846.264661
9443,2302122215,False,2.896780e-06,2023-02-12T22:13:23.721,2023-02-12 22:15:23.721,2023-02-12T22:48:03.725,1471,123,1,[2302124344],...,-318.151978,219.302628,0.562813,0.743522,-318.332275,216.011185,-48.874418,10.660818,-721.407299,252.699872
9444,2302122259,False,3.802894e-06,2023-02-12T22:53:23.725,2023-02-12 22:59:11.726,2023-02-12T23:11:27.727,1599,135,1,[2302124344],...,-255.484009,228.059631,0.445597,0.683996,-255.856812,221.742584,-45.531437,11.099579,-682.568534,264.721496


In [123]:
# can run this directly if len(final_df) < 5498

final_df["Earth X Helioprojective"] = earth_x_helioprojective 
final_df['Earth Y Helioprojective'] = earth_y_helioprojective

final_df["Lon Heliographic"] = earth_x_heliographic
final_df['Lat Heliographic'] = earth_y_heliographic

ValueError: Length of values (5498) does not match length of index (9446)

In [71]:
filtered = final_df[final_df['4-10 keV']>5*10**2] # shouldn't need to run this since we have already filtered for >500 couts in this case


In [72]:
filtered.to_csv('010121-130223_final_filtered_flare_list.csv')